In [1]:
%cd H-DenseUNet

[Errno 2] No such file or directory: 'H-DenseUNet'
/content


In [2]:
!git clone https://github.com/anhdhbn/H-DenseUNet.git
%cd H-DenseUNet
%mkdir /content/H-DenseUNet/data
%mkdir /content/H-DenseUNet/data/TrainingData
%mkdir /content/H-DenseUNet/data/TestData
!mkdir model
!pip install medpy

Cloning into 'H-DenseUNet'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 888 (delta 51), reused 42 (delta 20), pack-reused 807
Receiving objects: 100% (888/888), 13.79 MiB | 13.33 MiB/s, done.
Resolving deltas: 100% (361/361), done.
/content/H-DenseUNet
     |████████████████████████████████| 153kB 6.7MB/s 
     |████████████████████████████████| 42.5MB 1.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [0]:
# %cp -r "/content/drive/My Drive/Vef project/DenseNet161 Weights/densenet161_weights_tf.h5" "/content/H-DenseUNet/model/"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from __future__ import print_function
import sys
# sys.path.insert(0,'Keras-2.0.8')
from multiprocessing.dummy import Pool as ThreadPool
import random
from medpy.io import load
import numpy as np
import argparse
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from loss import weighted_crossentropy_2ddense
import os
import math

from keras.utils import multi_gpu_model

from denseunet import DenseUNet
from skimage.transform import resize
K.set_image_dim_ordering('tf')

from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
class A(object):
    pass
  

args = A()
args.data = "/content/drive/My Drive/Vef project/data"
args.save_path = "/content/H-DenseUNet/Experiments"
args.b = 4
args.input_size = 224
args.model_weight = './model/densenet161_weights_tf.h5'
args.input_cols = 3
args.mean = 48
args.thread_num = 14


MEAN = args.mean
thread_num = args.thread_num

liverlist = [32,34,38,41,47,87,89,91,105,106,114,115,119]

In [0]:
def load_seq_crop_data_masktumor_try(Parameter_List):
    img = Parameter_List[0]
    tumor = Parameter_List[1]
    lines = Parameter_List[2]
    numid = Parameter_List[3]
    minindex = Parameter_List[4]
    maxindex = Parameter_List[5]
    #  randomly scale
    scale = np.random.uniform(0.8,1.2)
    deps = int(args.input_size * scale)
    rows = int(args.input_size * scale)
    cols = 3

    sed = np.random.randint(1,numid)
    cen = lines[sed-1]
    cen = np.fromstring(cen, dtype=int, sep=' ')

    a = min(max(minindex[0] + deps/2, cen[0]), maxindex[0]- deps/2-1)
    b = min(max(minindex[1] + rows/2, cen[1]), maxindex[1]- rows/2-1)
    c = min(max(minindex[2] + cols/2, cen[2]), maxindex[2]- cols/2-1)
    cropp_img = img[a - deps / 2:a + deps / 2, b - rows / 2:b + rows / 2,
                c - cols / 2: c + cols / 2 + 1].copy()
    cropp_tumor = tumor[a - deps / 2:a + deps / 2, b - rows / 2:b + rows / 2,
                  c - cols / 2:c + cols / 2 + 1].copy()

    cropp_img -= MEAN
     # randomly flipping
    flip_num = np.random.randint(0, 8)
    if flip_num == 1:
        cropp_img = np.flipud(cropp_img)
        cropp_tumor = np.flipud(cropp_tumor)
    elif flip_num == 2:
        cropp_img = np.fliplr(cropp_img)
        cropp_tumor = np.fliplr(cropp_tumor)
    elif flip_num == 3:
        cropp_img = np.rot90(cropp_img, k=1, axes=(1, 0))
        cropp_tumor = np.rot90(cropp_tumor, k=1, axes=(1, 0))
    elif flip_num == 4:
        cropp_img = np.rot90(cropp_img, k=3, axes=(1, 0))
        cropp_tumor = np.rot90(cropp_tumor, k=3, axes=(1, 0))
    elif flip_num == 5:
        cropp_img = np.fliplr(cropp_img)
        cropp_tumor = np.fliplr(cropp_tumor)
        cropp_img = np.rot90(cropp_img, k=1, axes=(1, 0))
        cropp_tumor = np.rot90(cropp_tumor, k=1, axes=(1, 0))
    elif flip_num == 6:
        cropp_img = np.fliplr(cropp_img)
        cropp_tumor = np.fliplr(cropp_tumor)
        cropp_img = np.rot90(cropp_img, k=3, axes=(1, 0))
        cropp_tumor = np.rot90(cropp_tumor, k=3, axes=(1, 0))
    elif flip_num == 7:
        cropp_img = np.flipud(cropp_img)
        cropp_tumor = np.flipud(cropp_tumor)
        cropp_img = np.fliplr(cropp_img)
        cropp_tumor = np.fliplr(cropp_tumor)

    cropp_tumor = resize(cropp_tumor, (args.input_size,args.input_size,args.input_cols), order=0, mode='edge', cval=0, clip=True, preserve_range=True)
    cropp_img   = resize(cropp_img, (args.input_size,args.input_size,args.input_cols), order=3, mode='constant', cval=0, clip=True, preserve_range=True)
    return cropp_img, cropp_tumor[:,:,1]

def generate_arrays_from_file(batch_size,trainidx):
    while 1:
        X = np.zeros((batch_size, args.input_size, args.input_size, args.input_cols), dtype='float32')
        Y = np.zeros((batch_size, args.input_size, args.input_size, 1), dtype='int16')
        Parameter_List = []
        for idx in xrange(batch_size):
            count = random.choice(trainidx)

            img, img_header = load(args.data+ '/myTrainingData/volume-' + str(count) + '.nii')
            tumor, tumor_header = load(args.data + '/myTrainingData/segmentation-' + str(count) + '.nii')
            maxmin = np.loadtxt(args.data + '/myTrainingDataTxt/LiverBox/box_' + str(count) + '.txt', delimiter=' ')

            minindex = maxmin[0:3]
            maxindex = maxmin[3:6]
            minindex = np.array(minindex, dtype='int')
            maxindex = np.array(maxindex, dtype='int')
            minindex[0] = max(minindex[0] - 3, 0)
            minindex[1] = max(minindex[1] - 3, 0)
            minindex[2] = max(minindex[2] - 3, 0)
            maxindex[0] = min(img.shape[0], maxindex[0] + 3)
            maxindex[1] = min(img.shape[1], maxindex[1] + 3)
            maxindex[2] = min(img.shape[2], maxindex[2] + 3)
            
            f1 = open(args.data + '/myTrainingDataTxt/TumorPixels/tumor_' + str(idx) + '.txt', 'r')
            tumorline = f1.readlines()
            f1.close()
            
            f2 = open(args.data + '/myTrainingDataTxt/LiverPixels/liver_' + str(idx) + '.txt', 'r')
            liverline = f2.readlines()
            f2.close()
            
            num = np.random.randint(0,6)
            if num < 3 or (count in liverlist):
                lines = liverline
                numid = len(liverline)
            else:
                lines = tumorline
                numid = len(tumorline)
            Parameter_List.append([img, tumor, lines, numid, minindex, maxindex])
        pool = ThreadPool(thread_num)
        result_list = pool.map(load_seq_crop_data_masktumor_try, Parameter_List)
        pool.close()
        pool.join()
        for idx in xrange(len(result_list)):
            X[idx, :, :, :] = result_list[idx][0]
            Y[idx, :, :, 0] = result_list[idx][1]
        # print("getsizeof X, y", sys.getsizeof(X)*1.0/1024/1024, sys.getsizeof(Y)*1.0/1024/1024)
        yield (X,Y)


In [0]:
def load_fast_files(args, trainidx):  
    img_list = []
    tumor_list = []
    minindex_list = []
    maxindex_list = []
    tumorlines = []
    tumoridx = []
    liveridx = []
    liverlines = []
    number_sample = 0
    for idx in trainidx:
        print(idx)
        img, img_header = load(args.data+ '/myTrainingData/volume-' + str(idx) + '.nii')
        (x, y, z) = img.shape
        number_sample = int(z) + int(number_sample)
    return number_sample

In [0]:
    args.b = 3
    number_train = 11
    trainidx = list(range(number_train))
    validx = list(range(31,  33))
    number_samples = load_fast_files(args, trainidx)
    number_samples_val = load_fast_files(args, validx)

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)

    

    model = DenseUNet(reduction=0.5, args=args,nb_dense_block=4, growth_rate=12,nb_filter=24)
    # model.load_weights(args.model_weight, by_name=True)
    # model = make_parallel(model, args.b / 10, mini_batch=10)
    # model = multi_gpu_model(model, gpus=None)
    sgd = SGD(lr=1e-3, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss=[weighted_crossentropy_2ddense], metrics=['accuracy'])
#     model.summary()
    

    print('-'*30)
    print('Fitting model......')
    print('-'*30)

    if not os.path.exists(args.save_path):
        os.mkdir(args.save_path)

    if not os.path.exists(args.save_path + "/model"):
        os.mkdir(args.save_path + '/model')
        os.mkdir(args.save_path + '/history')
    else:
        if os.path.exists(args.save_path+ "/history/lossbatch.txt"):
            os.remove(args.save_path + '/history/lossbatch.txt')
        if os.path.exists(args.save_path + "/history/lossepoch.txt"):
            os.remove(args.save_path + '/history/lossepoch.txt')

    model_checkpoint = ModelCheckpoint(args.save_path + '/model/weights.{epoch:02d}-{loss:.2f}.hdf5', monitor='loss', verbose = 1,
                                       save_best_only=False,save_weights_only=False,mode = 'min', period = 1)


#     steps = math.ceil(number_samples / args.b)
    steps = 150
    steps_val = math.ceil(number_samples_val / args.b)
    print("steps", int(steps))
    print("steps_val", int(steps_val))
    # model.fit_generator(generate_arrays_from_file(args.b, trainidx, img_list, tumor_list, tumorlines, liverlines, tumoridx,
    #                                               liveridx, minindex_list, maxindex_list),steps_per_epoch=steps,
    #                                                 epochs= 6000, verbose = 1, callbacks = [model_checkpoint], max_queue_size=10,
    #                                                 workers=3, use_multiprocessing=True)
    history = model.fit_generator(generate_arrays_from_file(args.b, trainidx),steps_per_epoch=int(steps),validation_data=generate_arrays_from_file(args.b, validx),
                                                    validation_steps=steps_val,
                                                    epochs= 3, verbose = 1, callbacks = [model_checkpoint], max_queue_size=10,
                                                    workers=1, use_multiprocessing=False)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    print ('Finised Training .......')

0
1
2
3
4
5
6
7
8
9
10
31
32


W0711 07:20:32.778115 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 07:20:32.865375 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 07:20:32.917412 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 07:20:32.918646 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0711 07:20:32.924813 140596558

------------------------------
Creating and compiling model...
------------------------------


W0711 07:20:36.397550 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0711 07:20:36.511140 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0711 07:20:37.981086 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0711 07:20:55.549961 140596558296960 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0711 07:20:56.0

------------------------------
Fitting model......
------------------------------
steps 150
steps_val 76
Epoch 1/3


/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


150/150 [==============================] - 1951s 13s/step - loss: 0.4678 - acc: 0.9715 - val_loss: 0.3638 - val_acc: 1.0000

Epoch 00001: saving model to /content/H-DenseUNet/Experiments/model/weights.01-0.47.hdf5
Epoch 2/3
 46/150 [========>.....................] - ETA: 17:13 - loss: 0.3588 - acc: 1.0000

In [0]:
history.history['acc']